In [11]:
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma 
from langchain_openai import AzureOpenAIEmbeddings
from pathlib import Path
from chromadb.utils import embedding_functions
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document
import os


### load pdf

In [14]:
load_dotenv()
knowledge_base_paths = Path("KnowledgeBase")
files_path = [f for f in knowledge_base_paths.iterdir() if f.is_file()]
print(files_path)

docs = [PyPDFLoader(path).load() for path in files_path]
docs_list = [item for sublist in docs for item in sublist]
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs_list)
len(chunks)


[PosixPath('KnowledgeBase/verbraucherInformation.pdf')]


457

### load json

In [15]:
import json

# Dosya yolu
file_path = './KnowledgeBase/json/HannoverscheVerscicherung.json'

# JSON dosyasını okuma
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)



In [16]:
def create_Doc(page_content, source =file_path, page=0 ):
    doc = Document(
    metadata={'source': source, 'page': page},
    page_content=page_content
)
    return doc

In [17]:
for d in data:
    content = ''
    content += d['frage']
    content += d['ref_antwort']
    Doc = create_Doc(content)
    chunks.append(Doc)
len(chunks)

505

In [12]:
deployment = os.getenv("EMBEDDING_DEPLOYMENT")  
embedding_function = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    deployment=deployment)

In [20]:
import time

requests_per_minute = 600
sleep_time = 60 / requests_per_minute  # İstekler arasına bekleme süresi ekle

for chunk in chunks:
    vectorstore = Chroma.from_documents(
        documents=[chunk],  # Tek tek ekleyerek limiti aşmayı önle
        embedding=embedding_function,
        collection_name="hannoversche-versicherung",
        persist_directory="./.chromadb",
        collection_metadata={"embeding_model": "azure-openai-ada-002",
                             "chunk_size": 1000,
                             "chunk_overlap": 200}
    )
    time.sleep(sleep_time)  # Rate limit aşımını önlemek için bekleme süresi ekle


In [21]:
vectorstore._collection.count()

505

In [22]:
len(vectorstore._collection.peek(1)['embeddings'][0])

1536

In [24]:
vectorstore = Chroma( 
    collection_name="hannoversche-versicherung", 
    persist_directory="./.chromadb",
    collection_metadata={"embeding_model":"azure-openai-ada-002",
"chunk_size":1000,
"chunk_overlap":200})

retriever = vectorstore.as_retriever()